<a href="https://colab.research.google.com/github/Qv-code-Qv/kayak_data/blob/main/booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
!python booking.py

In [1]:
import plotly.express as px
import pandas as pd

In [4]:
df = pd.read_csv('gps_data_1.csv')
df.head()
   
fig = px.scatter_mapbox(df, opacity=1, title="Our best destinations for the coming week :", color="Ville", text="Hotel",  lat="Latitude", lon="Longitude",
                        zoom=4, mapbox_style="open-street-map")
fig.show()

In [18]:
import os 
import logging
import boto3.session
import my_key as key
import pandas as pd
import logging
import boto3
import sqlite3

AWS_ACCESS_KEY_ID = key.aws_access_key_id
AWS_SECRET_ACCESS_KEY =key.aws_secret_access_key
REGION_NAME = "eu-west-3"
BUCKET_NAME ="jehda-2024-ps-kayak.project"

booking_json = open('fichier_final/booking.json', 'rb')

#sauvegarde du résultat dans data lake (aws-s3)
logging.getLogger('botocore').setLevel(logging.INFO)
logging.getLogger('boto3').setLevel(logging.INFO)
session =   boto3.Session(aws_access_key_id = AWS_ACCESS_KEY_ID,
                        aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                        region_name= REGION_NAME)                  
s3_kayak = session.resource("s3")

""" CREATE BUCKET AND UPLOAD FILE """
bucket_kayak = s3_kayak.create_bucket(Bucket=BUCKET_NAME,
                          CreateBucketConfiguration={'LocationConstraint':REGION_NAME})
print("Création du bucket Jedha dans AWS :", bucket_kayak, "\n")
bucket_kayak.upload_file("fichier_final/booking.json","files/booking.json")

""" DOWNLOAD JSON FILE FROM S3 AND READ AS  DATAFRAME """
s3_obj = s3_kayak.Bucket(BUCKET_NAME).Object("files/booking.json").get()
booking_df = pd.read_json(s3_obj['Body'])
print(booking_df.head())

#CREATE Database booking
conn = sqlite3.connect(':memory:')
c= conn.cursor()

#CREATE TABLE booking
booking_df.to_sql('booking', conn, if_exists='replace', index = False)
c.execute('SELECT ville, name, score FROM booking LIMIT 5')
for row in c.fetchall():
    print (row)
conn.close()

""" SUPPRESSION JSON FILE in AWS  """
response = bucket_kayak.delete_objects(
    Delete={
        'Objects': [
            {
                'Key': "files/booking.json"
            }
        ]
    }
)
print ("\nconfirmation suppression booking.json file dans AWS", response, "\n")

response = bucket_kayak.delete()
print ("confirmation suppression bucket jedha-bucketdeletion dans AWS", response, "\n")

Création du bucket Jedha dans AWS : s3.Bucket(name='jehda-2024-ps-kayak.project') 

       ville                     name  \
0  Montauban                L'Amarelo   
1      Paris  Résidence Palais Étoile   
2      Paris               Hotel Zora   
3  Montauban                Le Studio   

                                             website    score  \
0  https://www.booking.com/hotel/fr/amarelo.fr.ht...  2 sur 5   
1  https://www.booking.com/hotel/fr/residence-pal...  1 sur 5   
2  https://www.booking.com/hotel/fr/zora.fr.html?...  1 sur 5   
3  https://www.booking.com/hotel/fr/le-studio-mon...  1 sur 5   

                                                desc        lat       lon  
0                Hébergement géré par un particulier  44.021308  1.351224  
1  Située à Paris, la Résidence Palais Étoile pro...  48.878389  2.284672  
2  Situé à 800 mètres du centre Pompidou, l'Hotel...  48.866459  2.346741  
3                Hébergement géré par un particulier  44.021355  1.351288  
('Mo